# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe0f1b12eb0>
├── 'a' --> tensor([[ 1.5487,  0.1930,  0.9436],
│                   [-1.6318,  2.1442, -0.5318]])
└── 'x' --> <FastTreeValue 0x7fe0f1b12fa0>
    └── 'c' --> tensor([[ 0.9625,  1.5090, -1.3196,  0.9571],
                        [ 1.3964, -1.2707,  0.6654, -1.2286],
                        [-0.5896, -0.0763,  0.2671,  1.0977]])

In [4]:
t.a

tensor([[ 1.5487,  0.1930,  0.9436],
        [-1.6318,  2.1442, -0.5318]])

In [5]:
%timeit t.a

84.8 ns ± 0.309 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe0f1b12eb0>
├── 'a' --> tensor([[ 0.4321, -1.6081,  0.7341],
│                   [-0.8887,  0.5625,  0.9779]])
└── 'x' --> <FastTreeValue 0x7fe0f1b12fa0>
    └── 'c' --> tensor([[ 0.9625,  1.5090, -1.3196,  0.9571],
                        [ 1.3964, -1.2707,  0.6654, -1.2286],
                        [-0.5896, -0.0763,  0.2671,  1.0977]])

In [7]:
%timeit t.a = new_value

91.4 ns ± 0.0391 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.5487,  0.1930,  0.9436],
               [-1.6318,  2.1442, -0.5318]]),
    x: Batch(
           c: tensor([[ 0.9625,  1.5090, -1.3196,  0.9571],
                      [ 1.3964, -1.2707,  0.6654, -1.2286],
                      [-0.5896, -0.0763,  0.2671,  1.0977]]),
       ),
)

In [10]:
b.a

tensor([[ 1.5487,  0.1930,  0.9436],
        [-1.6318,  2.1442, -0.5318]])

In [11]:
%timeit b.a

77.5 ns ± 0.129 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1726, -0.1748,  0.6979],
               [-0.4183, -2.4238,  1.0296]]),
    x: Batch(
           c: tensor([[ 0.9625,  1.5090, -1.3196,  0.9571],
                      [ 1.3964, -1.2707,  0.6654, -1.2286],
                      [-0.5896, -0.0763,  0.2671,  1.0977]]),
       ),
)

In [13]:
%timeit b.a = new_value

660 ns ± 0.311 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.07 µs ± 1.05 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.7 µs ± 14.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

166 µs ± 591 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

168 µs ± 1.01 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe0172e7940>
├── 'a' --> tensor([[[ 1.5487,  0.1930,  0.9436],
│                    [-1.6318,  2.1442, -0.5318]],
│           
│                   [[ 1.5487,  0.1930,  0.9436],
│                    [-1.6318,  2.1442, -0.5318]],
│           
│                   [[ 1.5487,  0.1930,  0.9436],
│                    [-1.6318,  2.1442, -0.5318]],
│           
│                   [[ 1.5487,  0.1930,  0.9436],
│                    [-1.6318,  2.1442, -0.5318]],
│           
│                   [[ 1.5487,  0.1930,  0.9436],
│                    [-1.6318,  2.1442, -0.5318]],
│           
│                   [[ 1.5487,  0.1930,  0.9436],
│                    [-1.6318,  2.1442, -0.5318]],
│           
│                   [[ 1.5487,  0.1930,  0.9436],
│                    [-1.6318,  2.1442, -0.5318]],
│           
│                   [[ 1.5487,  0.1930,  0.9436],
│                    [-1.6318,  2.1442, -0.5318]]])
└── 'x' --> <FastTreeValue 0x7fe0172e7a00>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38 µs ± 178 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe0eca19520>
├── 'a' --> tensor([[ 1.5487,  0.1930,  0.9436],
│                   [-1.6318,  2.1442, -0.5318],
│                   [ 1.5487,  0.1930,  0.9436],
│                   [-1.6318,  2.1442, -0.5318],
│                   [ 1.5487,  0.1930,  0.9436],
│                   [-1.6318,  2.1442, -0.5318],
│                   [ 1.5487,  0.1930,  0.9436],
│                   [-1.6318,  2.1442, -0.5318],
│                   [ 1.5487,  0.1930,  0.9436],
│                   [-1.6318,  2.1442, -0.5318],
│                   [ 1.5487,  0.1930,  0.9436],
│                   [-1.6318,  2.1442, -0.5318],
│                   [ 1.5487,  0.1930,  0.9436],
│                   [-1.6318,  2.1442, -0.5318],
│                   [ 1.5487,  0.1930,  0.9436],
│                   [-1.6318,  2.1442, -0.5318]])
└── 'x' --> <FastTreeValue 0x7fe0eca19070>
    └── 'c' --> tensor([[ 0.9625,  1.5090, -1.3196,  0.9571],
                        [ 1.3964, -1.2707,  0.6654, -1.2286],
                 

In [23]:
%timeit t_cat(trees)

35.8 µs ± 63.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77.7 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.9625,  1.5090, -1.3196,  0.9571],
                       [ 1.3964, -1.2707,  0.6654, -1.2286],
                       [-0.5896, -0.0763,  0.2671,  1.0977]],
              
                      [[ 0.9625,  1.5090, -1.3196,  0.9571],
                       [ 1.3964, -1.2707,  0.6654, -1.2286],
                       [-0.5896, -0.0763,  0.2671,  1.0977]],
              
                      [[ 0.9625,  1.5090, -1.3196,  0.9571],
                       [ 1.3964, -1.2707,  0.6654, -1.2286],
                       [-0.5896, -0.0763,  0.2671,  1.0977]],
              
                      [[ 0.9625,  1.5090, -1.3196,  0.9571],
                       [ 1.3964, -1.2707,  0.6654, -1.2286],
                       [-0.5896, -0.0763,  0.2671,  1.0977]],
              
                      [[ 0.9625,  1.5090, -1.3196,  0.9571],
                       [ 1.3964, -1.2707,  0.6654, -1.2286],
                       [-0.5896, -0.0763,  0.2671,  1.0977]],

In [26]:
%timeit Batch.stack(batches)

98 µs ± 607 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.9625,  1.5090, -1.3196,  0.9571],
                      [ 1.3964, -1.2707,  0.6654, -1.2286],
                      [-0.5896, -0.0763,  0.2671,  1.0977],
                      [ 0.9625,  1.5090, -1.3196,  0.9571],
                      [ 1.3964, -1.2707,  0.6654, -1.2286],
                      [-0.5896, -0.0763,  0.2671,  1.0977],
                      [ 0.9625,  1.5090, -1.3196,  0.9571],
                      [ 1.3964, -1.2707,  0.6654, -1.2286],
                      [-0.5896, -0.0763,  0.2671,  1.0977],
                      [ 0.9625,  1.5090, -1.3196,  0.9571],
                      [ 1.3964, -1.2707,  0.6654, -1.2286],
                      [-0.5896, -0.0763,  0.2671,  1.0977],
                      [ 0.9625,  1.5090, -1.3196,  0.9571],
                      [ 1.3964, -1.2707,  0.6654, -1.2286],
                      [-0.5896, -0.0763,  0.2671,  1.0977],
                      [ 0.9625,  1.5090, -1.3196,  0.9571],
                   

In [28]:
%timeit Batch.cat(batches)

181 µs ± 433 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

406 µs ± 6.08 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
